<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background- padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Total Knee Replacement: Attribution Analysis
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>
<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial'>Attribution modelling techniques aim to determine the contribution of each touchpoint or channel in a path or journey of healthcare decisions. Determining the importance of each touchpoint can aid in influencing the need for the necessary treatments. Using the touchpoints to create models can provide valuable insights into the effectiveness of the paths, which in turn will help in making informed decisions regarding the various treatments procedures that can be considered. With Teradata Vantage and ClearScape Analytics, users can get a full picture of these touchpoints.  Using pathing analytics, businesses can understand the common paths that lead to a variety of outcomes, such as Total Knee Replacement.</p>

<p style = 'font-size:18px;font-family:Arial'><b>Business Value</b></p>
<li style = 'font-size:16px;font-family:Arial'>Comprehensive health predictions and a reduced number of false positive and false negative results.</li>
<li style = 'font-size:16px;font-family:Arial'>Reduced cost to patients and hospitals.</li>
<li style = 'font-size:16px;font-family:Arial'>Identify patterns and symptoms leading to total knee replacements.</li>
<li style = 'font-size:16px;font-family:Arial'>Provides an understanding of various touchpoints that lead to total Knee Replacement.</li></p>

<p style = 'font-size:18px;font-family:Arial'><b>Why Vantage?</b></p>
<p style = 'font-size:16px;font-family:Arial'>Teradata Vantage provides a variety of attribution modeling including rule-based, statistical, and algorithmic-based attribution. Each has their own strengths for a variety of team across an organization. and limitations, while being useful across an organization. Thus, implementing an effective attribution model, using Teradata Vantage, can significantly enhance decision-making and optimize strategies.</p>
<p style = 'font-size:16px;font-family:Arial'>Also, ClearScape Analytics provides powerful, flexible attribution analysis, text processing, and statistical analytic techniques that can be applied to millions or billions of touchpoints. These results can be combined with other analytics to create more accurate models.</p>
<p style = 'font-size:16px;font-family:Arial'>Plus, Vantage allows organizations to scale these models horizontally (train segmented models per region, user type, etc.) or vertically (combine data from millions or billions of interactions). These models can be deployed operationally to understand and predict actions in real-time.</p>


<p style = 'font-size:16px;font-family:Arial'>In this script, we will apply the `ATTRIBUTION` function on a pathing dataset using all four prepared attribution models that define the weight of events in a sequence. Below are the types of Attribution models we will be using.</p>



 <table style = 'font-size:16px;font-family:Arial'>
  <tr>
    <th>Distribution Model</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>'LAST_CLICK'</td>
    <td>Conversion event is attributed entirely to most recent attributable event.</td>  
  </tr>
  <tr>
    <td>'FIRST_CLICK'</td>
    <td>Conversion event is attributed entirely to first attributable event.</td>  
  </tr>
  <tr>
    <td>'UNIFORM'</td>
    <td>Conversion event is attributed uniformly to preceding attributable events.</td>  
  </tr>
  <tr>
    <td>'EXPONENTIAL'</td>
    <td>Conversion event is attributed exponentially to preceding attributable events (the more recent the event, the higher the attribution).</td>  
  </tr>
</table>


<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>1. Connect to Vantage </b></p>
<p style = 'font-size:16px;font-family:Arial'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell. </p>

In [ ]:
%connect local, hidewarnings=true

<p style = 'font-size:16px;font-family:Arial'> We will setup for execution of notebook. Begin running steps with Shift + Enter keys.</p>


In [ ]:
Set query_band='DEMO=Knee_Replacement_Attribution_SQL.ipynb;' update for session;

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>2. Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>


In [ ]:
call get_data('DEMO_KneeReplacement_cloud'); ---takes about 10 seconds
---call get_data('DEMO_KneeReplacement_local'); ---takes about 30 seconds

<p style = 'font-size:16px;font-family:Arial'>Optional step – if you want to see status of databases/tables created and space used.</p>


In [ ]:
call space_report();  -- optional, takes about 10 seconds

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>3. Create Some Attribution Models</b></p>

<p style = 'font-size:16px;font-family:Arial'>In order to apply the `ATTRIBUTION` function on a pathing dataset, first we need to specify an attribution model. An attribution model defines how we are going to weight the activities that precede an event. </p>

<p style = 'font-size:16px;font-family:Arial'>There are five model types that we can use: 
<li style = 'font-size:16px;font-family:Arial'>*first event model*: the first event gets 100% of the weight </li>
<li style = 'font-size:16px;font-family:Arial'>*last event model*: the last event gets 100% of the weight </li>
<li style = 'font-size:16px;font-family:Arial'>*uniform event model*: all events are given equal weight  </li>
<li style = 'font-size:16px;font-family:Arial'>*exponential model*: each subsequent event in a sequence is given a higher weight than the preceding activity</li>
<li style = 'font-size:16px;font-family:Arial'>*weighted model*: attributed to preceding attributable events with weights specified by PARAMETERS.SEGMENT_SECONDS (specified by 'rows:K&seconds:K' in WindowSize syntax element)</li></p>

<p style = 'font-size:16px;font-family:Arial'>The total weight of all activities should equal to 1 (100%). Below are some Create table and Insert statements which will create model tables which will be used in the Attribution function.</p>

<p style = 'font-size:18px;font-family:Arial'> <b> Model 1: First Click Model</b></p>

In [ ]:
CREATE TABLE attribution_model_first (
    id INT, 
    model VARCHAR(100) 
); 

In [ ]:
INSERT INTO attribution_model_first VALUES (0,'EVENT_REGULAR'); 
INSERT INTO attribution_model_first VALUES (1, 'ALL:1.0:FIRST_CLICK:NA'); 

<p style = 'font-size:18px;font-family:Arial'> <b> Model 2: Last Click Model </b></p>



In [ ]:
CREATE TABLE attribution_model_last (
    id INT,
    model VARCHAR(100) 
); 

In [ ]:
INSERT INTO attribution_model_last VALUES (0, 'EVENT_REGULAR'); 
INSERT INTO attribution_model_last VALUES (1, 'ALL:1.0:LAST_CLICK:NA'); 

<p style = 'font-size:18px;font-family:Arial'> <b> Model 3: Uniform Event Model</b></p>


In [ ]:
CREATE TABLE attribution_model_uniform (
    id INT, 
    model VARCHAR(100) 
); 

In [ ]:
INSERT INTO attribution_model_uniform VALUES (0, 'EVENT_REGULAR'); 
INSERT INTO attribution_model_uniform VALUES (1, 'ALL:1.0:UNIFORM:NA'); 

<p style = 'font-size:18px;font-family:Arial'> <b> Model 4: Exponential Event Model</b></p>


In [ ]:
CREATE TABLE attribution_model_exponential (
    id INT, 
    model VARCHAR(100) 
); 

In [ ]:
INSERT INTO attribution_model_exponential VALUES (0, 'EVENT_REGULAR'); 
INSERT INTO attribution_model_exponential VALUES (1, 'ALL:1.0:EXPONENTIAL:0.5,ROW'); 

<p style = 'font-size:18px;font-family:Arial'> <b> Create Event of Interest (EOI) Table</b></p>
<p style = 'font-size:16px;font-family:Arial'>Since here we are trying to analyze the events that led to Total Knee Replacement our final Event of interest will be the same. </p>

In [ ]:
CREATE TABLE ATTRIBUTION_EOI( 
    event_of_interest VARCHAR(100)
);

In [ ]:
INSERT INTO ATTRIBUTION_EOI VALUES ('TOTAL KNEE REPLACEMENT');  

<p style = 'font-size:16px;font-family:Arial'>Before moving ahead let's check the data in these model tables to verify that we have correct data. </p>

In [ ]:
SELECT * FROM attribution_model_first; 

In [ ]:
SELECT * FROM attribution_model_last; 

In [ ]:
SELECT * FROM attribution_model_uniform; 

In [ ]:
SELECT * FROM attribution_model_exponential; 

In [ ]:
SELECT * FROM ATTRIBUTION_EOI; 

<p style = 'font-size:14px;font-family:Arial'><b> *Caution*: If more than the 2 rows are inserted for any attribution model table, it will err. Drop the table and reinsert.</b></p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>4. Sessionize Function </b></p>
<p style = 'font-size:16px;font-family:Arial'>The Sessionize function maps each click in a session to a unique session identifier. A session is a sequence of clicks by one user that are separated by at most n seconds. The function is useful for both sessionization and detecting web crawler ("bot") activity. A typical use is to understand user browsing behavior on a web site.</p>
<p style = 'font-size:16px;font-family:Arial'>However, we've also found it useful to define a sequence of events along a longitudinal record of a member's patient journey. In this case, we will define a session as one day.</p>

<p style = 'font-size:16px;font-family:Arial'>one day = 60 seconds x 60 minutes x 24 hours = 86,400 seconds</p>


In [ ]:
CREATE MULTISET TABLE KneeSessions AS (
    SELECT 
        *
    FROM SESSIONIZE (
        ON(
            SELECT memberid, 
                diagcode, 
                proccode, 
                TRANSLATE(shortdesc USING UNICODE_TO_LATIN) as shortdesc,
                amount, 
                tstamp, 
                lastname, 
                firstname, 
                state, 
                email
            FROM DEMO_KneeReplacement.knee_replacement
        )
        PARTITION BY memberid
        ORDER BY tstamp
        USING
            TIMECOLUMN('tstamp')
            TIMEOUT(86400)
            EMITNULL('true')
    ) 
) WITH DATA;

<p style = 'font-size:18px;font-family:Arial'> <b>View Data</b></p>

In [ ]:
SELECT TOP 5 * FROM KneeSessions; 

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>5. Attribution.</b</p>
<p style = 'font-size:18px;font-family:Arial'> <b> Run the Attribution Function Using A Uniform Attribution Distribution.</b></p>

In [ ]:
SELECT TOP 5
    F.memberid, F.SESSIONID, F.tstamp, F.shortdesc as event,
    F.ATTRIBUTION AS attribution_score
FROM ATTRIBUTION (
         ON KneeSessions AS INPUT
         PARTITION BY memberid
         ORDER BY tstamp
         ON ATTRIBUTION_EOI AS CONVERSION DIMENSION
         ON ATTRIBUTION_MODEL_UNIFORM AS MODEL1 DIMENSION
         USING
             EventColumn ('shortdesc')
             TIMESTAMPCOLUMN ('tstamp')
             WINDOWSIZE('seconds:31536000') -- 5 years
     ) F
     order by 1,2,5;

<p style = 'font-size:16px;font-family:Arial'>The output of the Attribution function using Uniform Attribution assigns equal weight to each event in a particular path apart from the Event of Interest which is 'TOTAL KNEE REPLACEMENT' in this case.</p>
<p style = 'font-size:16px;font-family:Arial'> <b>Create Attribution Model Outputs Table</b></p>


<p style = 'font-size:16px;font-family:Arial'>Write Outputs of all Attribution Model Types for Comparison

In [ ]:
CREATE MULTISET TABLE ATTRIBUTION_MODEL_OUTPUT AS (
    SELECT F.memberid as MEMBERID, F.SESSIONID as EVENTORDER, 
        F.tstamp as datestamp, F.shortdesc as EVENT,
        F.ATTRIBUTION AS FIRST_CLICK_ATTRIBUTION,
        L.ATTRIBUTION AS LAST_CLICK_ATTRIBUTION,
        U.ATTRIBUTION AS UNIFORM_ATTRIBUTION,
        E.ATTRIBUTION AS EXPONENTIAL_ATTRIBUTION
    FROM ATTRIBUTION (
         ON KneeSessions AS INPUT
         PARTITION BY memberid
         ORDER BY tstamp
         ON ATTRIBUTION_EOI AS CONVERSION DIMENSION
         ON ATTRIBUTION_MODEL_FIRST AS MODEL1 DIMENSION
         USING
             EventColumn ('shortdesc')
             TIMESTAMPCOLUMN ('tstamp')
             WINDOWSIZE('seconds:31536000') -- 5 years
     ) F
    ,ATTRIBUTION (
         ON KneeSessions AS INPUT
         PARTITION BY memberid
         ORDER BY tstamp
         ON ATTRIBUTION_EOI AS CONVERSION DIMENSION
         ON ATTRIBUTION_MODEL_LAST AS MODEL1 DIMENSION
         USING
             EventColumn ('shortdesc')
             TIMESTAMPCOLUMN ('tstamp')
             WINDOWSIZE('seconds:31536000') -- 5 years
     ) L
    ,ATTRIBUTION (
         ON KneeSessions AS INPUT
         PARTITION BY memberid
         ORDER BY tstamp
         ON ATTRIBUTION_EOI AS CONVERSION DIMENSION
         ON ATTRIBUTION_MODEL_UNIFORM AS MODEL1 DIMENSION
         USING
             EventColumn ('shortdesc')
             TIMESTAMPCOLUMN ('tstamp')
             WINDOWSIZE('seconds:31536000') -- 5 years
     ) U
    ,ATTRIBUTION (
         ON KneeSessions AS INPUT
         PARTITION BY memberid
         ORDER BY tstamp
         ON ATTRIBUTION_EOI AS CONVERSION DIMENSION
         ON ATTRIBUTION_MODEL_EXPONENTIAL AS MODEL1 DIMENSION
         USING
             EventColumn ('shortdesc')
             TIMESTAMPCOLUMN ('tstamp')
             WINDOWSIZE('seconds:31536000') -- 5 years
     ) E
WHERE F.memberid = L.memberid
AND   F.memberid = U.memberid
AND   F.memberid = E.memberid
AND   F.tstamp = L.tstamp
AND   F.tstamp = U.tstamp
AND   F.tstamp = E.tstamp
AND   F.shortdesc = L.shortdesc
AND   F.shortdesc = U.shortdesc
AND   F.shortdesc = E.shortdesc
)
WITH DATA;

<p style = 'font-size:16px;font-family:Arial'><b>View Attribution Outputs for Single Member</b></p>

In [ ]:
SELECT * FROM ATTRIBUTION_MODEL_OUTPUT WHERE MEMBERID=350031 ORDER BY 2 ASC ;

<p style = 'font-size:16px;font-family:Arial'>The above output of the Attribution Models is as below: 
<li style = 'font-size:16px;font-family:Arial'>*first event model*: the first event gets 100% of the weight :-  In the above output for the given member 'OTH ARTHROTOMY-KNEE' is the first click attribution i.e. the first event.</li> 
<li style = 'font-size:16px;font-family:Arial'>*last event model*: the last event gets 100% of the weight :- As seen in the output 'KNEE JOINT BIOPSY' is the last click attribution i.e. the last event before the event of Interest.</li>
<li style = 'font-size:16px;font-family:Arial'> *uniform event model*: all events are given equal weight :- As in the Uniform distribution column each event for that particular member is given equal weight i.e. 1/(Numberofevents-1) will give the attribution for each event.</li>
<li style = 'font-size:16px;font-family:Arial'>*exponential model*: each subsequent event in a sequence is given a higher weight than the preceding activity :- the exponential attribution is lowest for the first event and highest for the last event.</li>
</p>


<p style = 'font-size:18px;font-family:Arial'>Sum up the attribution values for each event/model combination</p>
<p style = 'font-size:16px;font-family:Arial'>Totals for each event/model combination should equal 1.0 (100%) for each member.</p>

In [ ]:
SELECT TOP 5 MEMBERID
       ,SUM(FIRST_CLICK_ATTRIBUTION) AS FIRST_CLICK_TOTAL
       ,SUM(LAST_CLICK_ATTRIBUTION)  AS LAST_CLICK_TOTAL
       ,SUM(UNIFORM_ATTRIBUTION)     AS UNIFORM_TOTAL
       ,SUM(EXPONENTIAL_ATTRIBUTION) AS EXPONENTIAL_TOTAL
FROM ATTRIBUTION_MODEL_OUTPUT
GROUP BY MEMBERID; 

<p style = 'font-size:16px;font-family:Arial'>As seen in the above output the totals for each event/model combination is equal to/nearing 1.0 (100%) for each member.</p>
<p style = 'font-size:18px;font-family:Arial'>Sum attribution values for each event</p>

In [ ]:
SELECT EVENT
       ,SUM(FIRST_CLICK_ATTRIBUTION) AS FIRST_CLICK_ATTRIBUTION
       ,SUM(LAST_CLICK_ATTRIBUTION)  AS LAST_CLICK_ATTRIBUTION
       ,SUM(UNIFORM_ATTRIBUTION)     AS UNIFORM_ATTRIBUTION
       ,SUM(EXPONENTIAL_ATTRIBUTION) AS EXPONENTIAL_ATTRIBUTION
FROM ATTRIBUTION_MODEL_OUTPUT
GROUP BY EVENT
ORDER BY 3 DESC; 

<p style = 'font-size:16px;font-family:Arial'>The above output shows the values of each type of Attribution for EACH event that we have in the different PATHS and the totals for each event/model combination.</p>
<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>6. Visualization</b></p>
    <p style = 'font-size:18px;font-family:Arial'> <b>Visualize Using Data from the Last Query</p>

In [ ]:
%chart EVENT, FIRST_CLICK_ATTRIBUTION  , title="First Click Attribution", width = 600 

<p style = 'font-size:16px;font-family:Arial'>The above output shows the total of each type of event for FIRST CLICK ATTRIBUTION. It means that the event <b>RANGE OF MOTION TESTING</b> was the first event in maximum number of cases followed by SKEL XRAY - THIGH/KNEE/LEG.</p>

In [ ]:
%chart EVENT, LAST_CLICK_ATTRIBUTION, title="Last Click Attribution", width = 600

<p style = 'font-size:16px;font-family:Arial'>The above output shows the total of each type of event for LAST CLICK ATTRIBUTION i.e. the last event before TOTAL KNEE REPLACEMENT. The event <b>KNEE JOINT BIOPSY </b> was the last event in maximum number of cases followed by KNEE ARTHROSCOPY.</p>

In [ ]:
%chart EVENT, UNIFORM_ATTRIBUTION, title="Uniform Attribution", width = 600

<p style = 'font-size:16px;font-family:Arial'>The above output shows the total of each type of event for UNIFORM ATTRIBUTION i.e. the occurrence of each event when the weights are uniformly distributed. The event <b>PHYSICAL THERAPY NEC</b> shows the maximum number of occurrences followed by OTH ARTHROTOMY-KNEE.</p>

In [ ]:
%chart EVENT, EXPONENTIAL_ATTRIBUTION, title="Exponential Attribution" , width = 600

<p style = 'font-size:16px;font-family:Arial'>The above output shows the total of each type of event for EXPONENTIAL ATTRIBUTION i.e. the first event has minimum weight and the last event has maximum weight. The above chart shows that the events KNEE ARTHROSCOPY, KNEE JOINT BIOPSY and OTH ARTHROTOMY-KNEE show almost similar number of cases where these events have the highest weights.</p>

<p style = 'font-size:18px;font-family:Arial'><b>Conclusion </b></p>
<p style = 'font-size:16px;font-family:Arial'>Thus , we can see how attribution modelling techniques help us in determining the importance of each touchpoint and how it can aid in influencing the need for the necessary treatments. Using these touchpoints and creating models have provided valuable insights into the effectiveness of the paths, which helped in making informed decisions. With Teradata Vantage and ClearScape Analytics, users can get a full picture of these touchpoints.</p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>7. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>We need to clean up our work tables to prevent errors next time.</p>

In [ ]:
DROP TABLE attribution_model_first;

In [ ]:
DROP TABLE attribution_model_last; 

In [ ]:
DROP TABLE attribution_model_uniform;

In [ ]:
DROP TABLE attribution_model_exponential; 

In [ ]:
DROP TABLE ATTRIBUTION_EOI; 

In [ ]:
DROP TABLE KneeSessions;

In [ ]:
DROP TABLE ATTRIBUTION_MODEL_OUTPUT;  

<p style = 'font-size:18px;font-family:Arial'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>We will use the following code to clean up tables and databases created for this demonstration.</p>


In [ ]:
call remove_data('DEMO_KneeReplacement'); 

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>Resources</b>
<p style = 'font-size:16px;font-family:Arial'>Let’s look at the elements we have available for reference for this notebook:</p>

<p style = 'font-size:18px;font-family:Arial'><b>Filters:</b></p>
<li style = 'font-size:16px;font-family:Arial'><b>Industry:</b> HealthCare </li>
<li style = 'font-size:16px;font-family:Arial'><b>Functionality:</b> Path Analytics</li>
<li style = 'font-size:16px;font-family:Arial'><b>Use Case:</b> Total Knee Replacement</li></p>
<p style = 'font-size:18px;font-family:Arial'><b>Related Resources:</b></p>
<li style = 'font-size:16px;font-family:Arial'><a href='https://teradata.seismic.com/Link/Content/DCGBP9J9gjD288TPcG3HFgXDHDW8'>Broken Digital Journeys CX Solution Accelerator Demo via Python Video - External - SP004183</a></li>
<li style = 'font-size:16px;font-family:Arial'><a href='https://www.teradata.com/Blogs/Forecasting-COVID-19-Using-Teradata-Vantage'>Forecasting COVID-19 Using Teradata Vantage</a></li>
<li style = 'font-size:16px;font-family:Arial'><a href='https://usc-word-edit.officeapps.live.com/we/%E2%80%A2%09https:/www.teradata.com/Blogs/Predicting-Heart-Failure-with-Teradata'>Saving Lives, Saving Costs: Predicting Heart Failure with Teradata</a></li></p>



<footer style="padding-bottom:35px; background:#91A0AB; ">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023,2024. All Rights Reserved
        </div>
    </div>
</footer>